## **Mountain names recognition NER project**

Task: to train a named entity recognition (NER) model for the identification of mountain names inside the texts.

I will use pretrained BERT model (namely BertForTokenClassification) which can make predictions at token level, rather than at sequence level. Then I will fine-tune it on database consisting of sentences containing mountain names.

**Note: I'm going to provide an option of running my code in Google Colab using GPU for acceleration purposes.**

#### **Importing Python Libraries and preparing the environment**

In [1]:
!pip install transformers seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=fd1ad407a1535db49b18172a3b4727b71a622493735538505d05d9a6ff9be8ea
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

I set the default device to GPU using the following code (if it prints "cuda", it means the GPU has been recognized):

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


#### **Downloading and preprocessing the data**
NER uses a specific annotation scheme which is defined at word level. An annotation scheme that is widely used is called IOB-tagging which stands for Inside-Outside-Beginning. Each tag indicates whether the corresponding word is *inside*, *outside* or at the *beginning* of a specific named entity. The reason this is used is because named entities usually comprise more than 1 word. However, for the purpose of my project I use only 2 types of tags: "M" for mountains, "O" for all other words.

I used the list of all mountains from Wikipedia https://en.wikipedia.org/wiki/List_of_mountains_by_elevation. I chose 250 single-word mountains for simplification purpose and wrote prompts in ChatGPT in order to compose 250 sentences containing selected mountain names. I saved these sentences in Excel file.

In [4]:
pd.set_option('display.max_colwidth', None)
data = pd.read_excel("250_sentences.xlsx")

Below is the list of all mountains retrieved from Wikipedia and saved as dataframe.

In [5]:
list_of_mountains = pd.read_excel("list_of_all_mountains.xlsx")

Also, I need NLTK library to tokenize sentences in my database.

In [6]:
import nltk
nltk.download('all')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

The code below split sentences into tokens which are used for tagging words as mountains ("M") or not ("O").

In [7]:
tags = []
for i in range(0,len(data['Sentence'])):
    tags.append(','.join('M' if token in list_of_mountains['Mountain'].tolist() else 'O' for token in word_tokenize(data['Sentence'][i])))

data['Tag'] = tags
data.head()

,Sentence,Tag
0,"Nestled in the heart of the Himalayas, Everest stands as the pinnacle of Earth's grandeur.","O,O,O,O,O,O,O,O,M,O,O,O,O,O,O,O,O,O"
1,"Majestic slopes, like those found on K2, challenge even the most seasoned mountaineers, beckoning them with the awe-inspiring allure.","O,O,O,O,O,O,O,M,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,"Casting a tranquil shadow over the surrounding landscapes, Kangchenjunga boasts serene peaks.","O,O,O,O,O,O,O,O,O,M,O,O,O,O"
3,"Lhotse, with its formidable neighbor Everest, shares the same breathtaking vistas but remains a distinct entity.","M,O,O,O,O,O,M,O,O,O,O,O,O,O,O,O,O,O,O"
4,"Commanding respect as one of the world's formidable peaks, Makalu presents an icy countenance.","O,O,O,O,O,O,O,O,O,O,O,M,O,O,O,O,O"


Let's check how many sentences (and corresponding sequences of tags) there are in this dataset.

In [8]:
data.count()

Sentence    250
Tag         250
dtype: int64

I created 2 dictionaries: one that maps individual tags to indices, and one that maps indices to their individual tags.

In [9]:
labels_to_ids = {'O': 0, 'M': 1}
ids_to_labels = {0: 'O', 1: 'M'}
labels_to_ids

{'O': 0, 'M': 1}

In [10]:
# let's make dataframe more comfortable for future processing
data['sentence'] = data['Sentence']
data['word_labels'] = data['Tag']
data = data[["sentence", "word_labels"]]
data.head()

,sentence,word_labels
0,"Nestled in the heart of the Himalayas, Everest stands as the pinnacle of Earth's grandeur.","O,O,O,O,O,O,O,O,M,O,O,O,O,O,O,O,O,O"
1,"Majestic slopes, like those found on K2, challenge even the most seasoned mountaineers, beckoning them with the awe-inspiring allure.","O,O,O,O,O,O,O,M,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,"Casting a tranquil shadow over the surrounding landscapes, Kangchenjunga boasts serene peaks.","O,O,O,O,O,O,O,O,O,M,O,O,O,O"
3,"Lhotse, with its formidable neighbor Everest, shares the same breathtaking vistas but remains a distinct entity.","M,O,O,O,O,O,M,O,O,O,O,O,O,O,O,O,O,O,O"
4,"Commanding respect as one of the world's formidable peaks, Makalu presents an icy countenance.","O,O,O,O,O,O,O,O,O,O,O,M,O,O,O,O,O"


In [11]:
len(data)

250

Let's verify that a random sentence and its corresponding tags are correct:

In [12]:
data.iloc[10].sentence

'Whispering tales of ancient legends and modern conquests, Himalchuli stands proud in the Manaslu region.'

In [13]:
data.iloc[10].word_labels

'O,O,O,O,O,O,O,O,O,M,O,O,O,O,M,O,O'

#### **Preparing the dataset and dataloader**

Data is preprocessed, so it's time to turn it into PyTorch tensors that can be fed to the model. Let's start by defining some key hyperparameters that will be used later on in the training and evaluation process.

In [14]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 3
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

A tricky part of NER with BERT is that BERT relies on wordpiece tokenization, rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level.

Below, I defined a regular PyTorch dataset class which transforms examples of a dataframe to PyTorch tensors. Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above. Word pieces that should be ignored have a label of -100 (default ignore_index of PyTorch's CrossEntropyLoss).

In [15]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(",")

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True,
                             return_offsets_mapping=True,
                             padding='max_length',
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

  def __len__(self):
        return self.len

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split:

In [16]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (250, 2)
TRAIN Dataset: (200, 2)
TEST Dataset: (50, 2)


Let's have a look at the first training example:

In [17]:
training_set[0]

{'input_ids': tensor([  101, 21153,  1010,  2019, 16625,  4672,  1999,  1996, 23277, 22043,
          3363,  4020,  1010,  2003,  1037,  4119,  2005,  7105,  2545,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

Let's verify that the input ids and corresponding targets are correct:

In [18]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
sanford     1
,           -100
an          0
imposing    0
peak        0
in          0
the         0
wr          0
##ange      -100
##ll        -100
mountains   1
,           -100
is          0
a           0
challenge   0
for         0
climb       0
##ers       -100
.           -100
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[P

Now, let's define the corresponding PyTorch dataloaders:

In [19]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased".

Note: only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights which will be trained, together with the pretrained weights using labelled dataset. This is also printed as a warning when you run the code cell below.

Then, I move the model to the GPU.

In [20]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

#### **Training the model**



Next, optimizer should be defined. Here, I am just going to use Adam with a default learning rate.

In [22]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a training function.

In [23]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        output_support = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output_support.loss.item()
        tr_logits = output_support.logits

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        output_support.loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss: {epoch_loss}")
    print(f"Training accuracy: {tr_accuracy}")

Then let's train the model.

In [24]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.6255170106887817
Training loss: 0.253419403731823
Training accuracy: 0.9150523721538439
Training epoch: 2
Training loss per 100 training steps: 0.18314525485038757
Training loss: 0.13936539337038995
Training accuracy: 0.9294586939654826
Training epoch: 3
Training loss per 100 training steps: 0.07158251851797104
Training loss: 0.08355541288852691
Training accuracy: 0.9726340414798881


#### **Evaluating the model**

Now we can evaluate model's performance on test dataset.

In [25]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

            output_for_valid = model(input_ids=ids, attention_mask=mask, labels=labels)
            eval_loss += output_for_valid.loss.item()
            eval_logits = output_for_valid.logits

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [id.item() for id in eval_labels]
    predictions = [id.item() for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation loss: {eval_loss}")
    print(f"Validation accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good. Accuracy on the test dataset is > 98%.

In [26]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.025587383657693863
Validation loss: 0.08166303604841232
Validation accuracy: 0.9842784427924681


However, the accuracy metric is misleading, as a lot of labels are not mountains (label "O"), even after omitting predictions on the [PAD] tokens. It's useful to look at the precision, recall and f1-score of the individual tags.

In [27]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       631
           1       0.98      0.81      0.89        54

    accuracy                           0.98       685
   macro avg       0.98      0.91      0.94       685
weighted avg       0.98      0.98      0.98       685



Performance seems to be quite good. It can be better if the model is trained for more epochs (in this project it was trained for 3 epochs).

#### **Inference**

The fun part is when we can quickly test the model on new, unseen sentences.

In [32]:
sentence = "Annapurna provides hikers with a number of amazing views."

inputs = tokenizer(sentence.split(),
                    is_split_into_words=True,
                    return_offsets_mapping=True,
                    padding='max_length',
                    truncation=True,
                    max_length=MAX_LEN,
                    return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, attention_mask=mask)
logits = outputs.logits

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.view(-1).tolist())
token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

prediction = []
for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].view(-1,2).tolist()):
  #only predictions on first word pieces are important
  if mapping[0] == 0 and mapping[1] != 0:
    prediction.append(token_pred[1])
  else:
    continue

print(sentence.split())
print(prediction)

['Annapurna', 'provides', 'hikers', 'with', 'a', 'number', 'of', 'amazing', 'views.']
['M', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


#### **Saving the model for future use**

In [33]:
import os

directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

All files saved
